In [ ]:
import os
import dask
from dask.distributed import Client
from azureml.core import Run

In [ ]:
def update_packages():
    os.system('pip install --upgrade fsspec adlfs')

In [ ]:
computes = [dask.delayed(update_packages)() for i in range(16)]
computes;

In [ ]:
run = Run.get_context()
run

c = Client(run.get_metrics()['scheduler'])
c.compute(computes)

In [1]:
import os
import dask
import time
#import adlfs
import joblib
import fsspec
import socket
import matplotlib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from dask.distributed import Client
from IPython.core.display import HTML
from dask_ml.xgboost import XGBRegressor

from azureml.core import Run

%matplotlib inline

In [2]:
run = Run.get_context()
run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-dask-demo,dask-dask-demo_1579797359_4a4da0f1,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
c = Client(run.get_metrics()['scheduler'])
c

Client Scheduler: tcp://10.3.0.5:8786 Dashboard: http://10.3.0.5:8787/status,Cluster Workers: 15 Cores: 15 Memory: 0 B


In [20]:
c.restart()

Client Scheduler: tcp://10.3.0.5:8786 Dashboard: http://10.3.0.5:8787/status,Cluster Workers: 15 Cores: 15 Memory: 0 B


In [ ]:
#from dask_cuda import LocalCUDACluster

In [ ]:
#cluster = LocalCUDACluster()

In [ ]:
#help(cluster.__init__)

In [ ]:
#c.restart()

In [4]:
#c = Client(cluster)
c

Client Scheduler: tcp://10.3.0.5:8786 Dashboard: http://10.3.0.5:8787/status,Cluster Workers: 15 Cores: 15 Memory: 0 B


In [21]:
STORAGE_OPTIONS = {
    'account_name': run.experiment.workspace.datastores['gen2'].account_name,
    'account_key' : run.experiment.workspace.datastores['gen2'].account_key
}

In [22]:
protocol   = 'abfs'
filesystem = 'datasets'

In [23]:
data_path = f'{protocol}://{filesystem}'
data_path

'abfs://datasets'

In [31]:
data_path = f'{run.get_metrics()["datastore"]}'
data_path

'/mnt/batch/tasks/shared/LS_root/jobs/dask-azureml/azureml/dask-dask-demo_1579797359_4a4da0f1/mounts/gen2'

In [32]:
#fs = fsspec.filesystem(protocol, **STORAGE_OPTIONS, container_name=filesystem)

In [33]:
#files = []
#for file in fs.glob('noaa/isd/year=*/month=*'): # see https://github.com/dask/adlfs/issues/34
#    files += fs.ls(f'{file}/')
#files = [f'{data_path}/{file}' for file in files if '2019' not in file] # issue with 2019 data - fix wip 
#files[-5:]

In [39]:
import glob

files = glob.glob(f'{data_path}/noaa/isd/year=*/month=*/*.parquet')

In [40]:
import cudf
import dask_cudf

In [ ]:
df = dask.delayed(dask_cudf.read_parquet)(f'{data_path}/noaa/isd/year=*/month=*/*.parquet').compute()#, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()
#df = dask_cudf.from_dask_dataframe(df)
#df = cudf.read_parquet(files, storage_options=STORAGE_OPTIONS)
%time df.head()

In [44]:
#df = df.set_index(dd.to_datetime(df.datetime).dt.floor('d'), sorted=False)
#df = df.persist() 
%time len(df)

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 15 µs


8103070

In [45]:
%time len(df)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 26.7 µs


8103070

In [47]:
%time df.describe()

CPU times: user 154 ms, sys: 100 ms, total: 254 ms
Wall time: 269 ms


,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,year,day,version
count,8.103070e+06,8.103070e+06,8.103070e+06,6.683238e+06,6.890551e+06,7.905611e+06,2.961762e+06,1.011276e+06,406960.000000,1.604593e+06,1.604593e+06,65899.000000,8103070.0,8.103070e+06,8103070.0
mean,3.800378e+01,-4.120244e+01,3.522812e+02,1.693299e+02,3.914847e+00,1.905801e+00,1.016950e+03,2.985789e+01,4.107468,5.416817e+00,3.708085e+01,26.130928,2008.0,1.608984e+01,1.0
std,2.081532e+01,7.861071e+01,5.371495e+02,1.131514e+02,3.319548e+00,1.300685e+01,1.278503e+01,2.740872e+01,2.820931,8.105268e+00,5.328411e+02,53.587699,0.0,8.929491e+00,0.0
min,-9.000000e+01,-1.796330e+02,-3.500000e+02,0.000000e+00,0.000000e+00,-6.550000e+01,9.113000e+02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,2008.0,1.000000e+00,1.0
25%,3.395600e+01,-9.661000e+01,3.000000e+01,7.000000e+01,1.500000e+00,-5.100000e+00,1.009800e+03,1.000000e+01,2.000000,1.000000e+00,0.000000e+00,4.000000,2008.0,8.000000e+00,1.0
50%,4.147800e+01,-7.881700e+01,1.630000e+02,1.800000e+02,3.100000e+00,2.000000e+00,1.017700e+03,1.000000e+01,4.000000,1.000000e+00,0.000000e+00,14.000000,2008.0,1.600000e+01,1.0
75%,4.830800e+01,1.261700e+01,3.900000e+02,2.600000e+02,5.700000e+00,9.000000e+00,1.024800e+03,6.100000e+01,7.000000,6.000000e+00,0.000000e+00,33.000000,2008.0,2.400000e+01,1.0
max,8.251700e+01,1.797500e+02,9.999000e+03,3.600000e+02,9.000000e+01,5.400000e+01,1.085000e+03,9.900000e+01,9.000000,9.900000e+01,9.999000e+03,999.000000,2008.0,3.100000e+01,1.0


In [ ]:
%time bites = df.memory_usage(index=True, deep=True).sum().compute()
print(f'Dataframe is: {round(bites/1e9, 2)}GB')

In [ ]:
%time means = df.groupby(df.index).mean().compute()
means.head()

In [ ]:
for col in list(means.columns):
    fig = plt.figure(figsize=(16, 8))
    #plt.style.use('dark_background')
    means[col].plot(color='b')
    plt.title('Average of {}'.format(col))
    plt.xlim([datetime(2008, 1, 1), datetime(2018, 12, 31)])
    plt.grid()
    
    # optionally, log the image to the run
    run.log_image(f'mean_{col}', plot=plt)

You can see the images logged to the run in the studio.

In [ ]:
run

## Prepare data

In [ ]:
## insert any Pandas-like Dask data prep code 
df['temperature'] = df['temperature']*(9/5)+32 # 'Merica

## Write data

**Important:** you will have not access to write to this account, get your own storage account! 

See the [API documentation for dask.dataframe.to_parquet](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.to_parquet) for additional settings. 

In [ ]:
df = df.repartition(npartitions=df.npartitions*10)
%time dask.delayed(df.to_parquet)(f'abfs://outputs/noaa/isd_out.parquet', compression='lz4', storage_options=STORAGE_OPTIONS).compute()

![Write gif](media/write.gif)

## Train XGBoost model

In [ ]:
df = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()

In [ ]:
files = []
for file in fs.glob('noaa/isd/year=*/month=*'): # see https://github.com/dask/adlfs/issues/34
    files += fs.ls(f'{file}/')
files = [f'{protocol}://{container}/{file}' for file in files if '2019' in file] 

df2019 = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute() 

In [ ]:
# begin data prep
df = df.fillna(0) 
df2019 = df2019.fillna(0) 

In [ ]:
df['month'] = df['datetime'].dt.month
df2019['month'] = df2019['datetime'].dt.month

In [ ]:
cols = list(df.columns)
cols = [col for col in cols if df.dtypes[col] != 'object' and col not in ['version', 'datetime']]
cols

In [ ]:
X = df[[col for col in cols if col not in ['temperature']]].persist()
y = df.temperature.persist()
# end data prep - persist intelligently per https://docs.dask.org/en/latest/best-practices.html

In [ ]:
xgb = XGBRegressor(n_estimators=16)
%time xgb.fit(X, y)

In [ ]:
%time y_pred = xgb.predict(X).compute()

In [ ]:
rmse = (((y.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Training RMSE: {round(rmse, 3)}')

## Evaluate model

In [ ]:
X_test = df2019[[col for col in cols if col not in ['temperature']]].persist()
y_test = df2019.temperature.persist()

In [ ]:
%time y_pred = xgb.predict(X_test).compute()

In [ ]:
rmse = (((y_test.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Test RMSE: {round(rmse, 3)}')

## Register model

In [ ]:
model_path = 'xgboost_noaa_isd.joblib.dat'
joblib.dump(xgb, model_path)
xgb = joblib.load(model_path)

In [ ]:
model = Model.register(ws, model_path, 'xgboost-noaa-isd', 
                       description='Dask XGBoost NOAA ISD temperature predictor',
                       model_framework='XGBoost')

## End the run

Cluster will return to 0 nodes.

In [ ]:
c.close()
run.cancel()

In [ ]:
t_end = time.time()
print(f'Total run time: {round((t_end-t_start)/60, 2)} minutes')